# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'02-18-2022'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-18-2022'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-02-19 04:21:11,33.93911,67.709953,171857,7526,NaN,NaN,Afghanistan,441.470136,4.379222
1,NaN,NaN,NaN,Albania,2022-02-19 04:21:11,41.15330,20.168300,269601,3432,NaN,NaN,Albania,9368.302175,1.272992
2,NaN,NaN,NaN,Algeria,2022-02-19 04:21:11,28.03390,1.659600,263685,6780,NaN,NaN,Algeria,601.319791,2.571250
3,NaN,NaN,NaN,Andorra,2022-02-19 04:21:11,42.50630,1.521800,37589,150,NaN,NaN,Andorra,48649.453181,0.399053
4,NaN,NaN,NaN,Angola,2022-02-19 04:21:11,-11.20270,17.873900,98605,1899,NaN,NaN,Angola,300.018852,1.925866


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/9/22,2/10/22,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,168550,169448,169940,170152,170604,171246,171422,171519,171673,171857
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,266416,267020,267020,267551,268008,268304,268491,268940,269301,269601
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,259673,260191,260723,261226,261752,262165,262570,262994,263369,263685


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/9/22,2/10/22,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7465,7477,7478,7488,7494,7501,7504,7513,7524,7526
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3399,3402,3402,3407,3410,3414,3420,3423,3430,3432
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6678,6690,6703,6715,6729,6744,6753,6762,6772,6780


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/9/22,2/10/22,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/9/22,2/10/22,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15121,15155,15216,15288,15307,15325,15350,15366,15387,15409
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,54095,54203,54301,54391,54444,54481,54545,54614,54663,54700
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5380,5391,5402,5408,5409,5411,5413,5413,5422,5424


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/9/22,2/10/22,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,173,173,173,176,176,176,177,181,181,183
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,626,626,626,626,626,626,629,630,631,634
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,85,87,87,89,89,89,90,91,91,92


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
674,1001.0,Autauga,Alabama,US,2022-02-19 04:21:11,32.539527,-86.644082,15409,183,NaN,NaN,"Autauga, Alabama, US",27580.590309,1.187618
711,1075.0,Lamar,Alabama,US,2022-02-19 04:21:11,33.779950,-88.096680,3586,63,NaN,NaN,"Lamar, Alabama, US",25976.095618,1.756832
712,1077.0,Lauderdale,Alabama,US,2022-02-19 04:21:11,34.901719,-87.656247,23722,359,NaN,NaN,"Lauderdale, Alabama, US",25582.072491,1.513363


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,78423462,934321,0.0
India,42780917,510912,0.0
Brazil,28072238,643340,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,78423462,934321,0.0,77489141.0,2022-02-19 04:21:11,37.934269,-91.444083
India,42780917,510912,0.0,42270005.0,2022-02-19 04:21:11,23.088275,81.806127
Brazil,28072238,643340,0.0,27428898.0,2022-02-19 04:21:11,-12.669522,-48.480493
France,22310014,137448,0.0,22172566.0,2022-02-19 04:21:11,6.430808,-34.730285
United Kingdom,18676361,160946,0.0,18515415.0,2022-02-19 04:21:11,30.654806,-30.248397


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,8888473,83862,0.0
Texas,6567203,83447,0.0
Florida,5818706,68902,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,8888473,83862,0.0,8804611.0,2022-02-19 04:21:11,37.843962,-120.728594
Texas,6567203,83447,0.0,6483756.0,2022-02-19 04:21:11,31.660643,-98.653069
Florida,5818706,68902,0.0,5749804.0,2022-02-19 04:21:11,28.940755,-82.700744
New York,4903700,66919,0.0,4836781.0,2022-02-19 04:21:11,42.544151,-75.474183
Illinois,3013709,34732,0.0,2978977.0,2022-02-19 04:21:11,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,2777165,30216,0.0
Arizona,Maricopa,1237579,15486,0.0
Florida,Miami-Dade,1173497,10153,0.0
Illinois,Cook,1105263,13884,0.0
Texas,Harris,986677,10540,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,2777165,30216,0.0,2746949.0,2022-02-19 04:21:11,34.308284,-118.228241,6037.0
Arizona,Maricopa,1237579,15486,0.0,1222093.0,2022-02-19 04:21:11,33.348359,-112.491815,4013.0
Florida,Miami-Dade,1173497,10153,0.0,1163344.0,2022-02-19 04:21:11,25.611236,-80.551706,12086.0
Illinois,Cook,1105263,13884,0.0,1091379.0,2022-02-19 04:21:11,41.841448,-87.816588,17031.0
Texas,Harris,986677,10540,0.0,976137.0,2022-02-19 04:21:11,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-16,171519,268940,262994,37452,98568,11,7395,8783208,410155,2967316,...,795316,234359,7,508042,2606824,622175,507,11707,310155,232213
2022-02-17,171673,269301,263369,37522,98585,11,7395,8799858,411878,2990045,...,800833,234600,7,508968,2643024,625028,507,11718,310474,232598
2022-02-18,171857,269601,263685,37589,98605,11,7400,8815247,413295,3013262,...,808012,234872,7,509947,2685463,627956,508,11731,310764,233030


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-16,7513,3423,6762,150,1899,0,135,124765,8229,4797,...,6820,1609,1,5564,39188,5320,0,2113,3939,5379
2022-02-17,7524,3430,6772,150,1899,0,135,124924,8253,4833,...,6838,1612,1,5572,39278,5339,0,2114,3941,5381
2022-02-18,7526,3432,6780,150,1899,0,135,125062,8278,4878,...,6854,1614,1,5591,39358,5357,0,2115,3942,5385


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-02-17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-02-18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/9/22,2/10/22,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15121,15155,15216,15288,15307,15325,15350,15366,15387,15409
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,3514,3520,3527,3539,3540,3541,3547,3553,3577,3586
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,23366,23420,23502,23535,23583,23597,23633,23672,23698,23722
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,7327,7354,7384,7399,7417,7421,7433,7452,7470,7486
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,37018,37059,37125,37243,37267,37282,37319,37351,37382,37415


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-16,1267907,234810,18,1957085,810520,8865246,1297330,717520,254083,49,...,1979762,6545752,916191,110171,15229,1616749,1410611,479214,1561035,153663
2022-02-17,1270015,234810,18,1959866,811669,8879455,1299468,718064,254441,49,...,1983621,6555490,917223,110417,15255,1619839,1410499,480692,1562836,153850
2022-02-18,1271455,235956,18,1962920,812948,8888473,1301611,719327,254719,49,...,1987450,6567203,918222,110679,15255,1622685,1410499,481818,1564381,154082


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-02-16       15366   54614    5413  6336  14594    2270   5006   31267   
2022-02-17       15387   54663    5422  6345  14632    2271   5011   31302   
2022-02-18       15409   54700    5424  6349  14643    2271   5017   31354   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-02-16         8388     5023  ...    6651   1871     7958     1908   
2022-02-17         8394     5032  ...    6660   1876     7964     1909   
2022-02-18         8399     5039  ...    6668   1880     7973     1910   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-02-16          10920  9642  5593          0     2297   1530  
2022-02-17          10926  9669  5598          0     2298   1531  
2022-02-18          10937  9692  5599          0     2303   1532  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-16,17749,1141,0,27189,10190,83366,11659,10318,2654,0,...,23466,82896,4336,575,107,17878,11522,6104,12991,1689
2022-02-17,17787,1141,0,27398,10235,83588,11675,10324,2655,0,...,23868,83193,4344,579,108,17947,11522,6146,13043,1689
2022-02-18,17877,1141,0,27513,10271,83862,11681,10357,2660,0,...,23910,83447,4359,583,108,18016,11522,6162,13068,1689


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-02-16         181     630      91   98    214      49    113     573   
2022-02-17         181     631      91   98    215      49    114     574   
2022-02-18         183     634      92   98    215      49    114     575   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-02-16          149       77  ...     131     39       72       27   
2022-02-17          149       77  ...     131     39       72       27   
2022-02-18          150       77  ...     131     39       72       27   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-02-16            122    15    36          0       42     17  
2022-02-17            122    15    36          0       42     17  
2022-02-18            122    15    36          0       42     17  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-16,0.000566,0.001672,0.001615,0.002436,0.000132,0.0,0.007219,0.001943,0.004344,-0.000867,...,0.008419,0.001265,0.0,0.002101,0.013505,0.005898,0.003960,0.000684,0.000920,0.002634
2022-02-17,0.000898,0.001342,0.001426,0.001869,0.000172,0.0,0.000000,0.001896,0.004201,0.007660,...,0.006937,0.001028,0.0,0.001823,0.013887,0.004586,0.000000,0.000940,0.001029,0.001658
2022-02-18,0.001072,0.001114,0.001200,0.001786,0.000203,0.0,0.000676,0.001749,0.003440,0.007765,...,0.008964,0.001159,0.0,0.001924,0.016057,0.004685,0.001972,0.001109,0.000934,0.001857


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-16,0.001199,0.000877,0.001333,0.0,0.0,NaN,0.007463,0.001445,0.003414,0.015023,...,0.003827,0.001245,0.0,0.001440,0.001687,0.004532,NaN,0.002848,0.000000,0.000930
2022-02-17,0.001464,0.002045,0.001479,0.0,0.0,NaN,0.000000,0.001274,0.002917,0.007505,...,0.002639,0.001865,0.0,0.001438,0.002297,0.003571,NaN,0.000473,0.000508,0.000372
2022-02-18,0.000266,0.000583,0.001181,0.0,0.0,NaN,0.000000,0.001105,0.003029,0.009311,...,0.002340,0.001241,0.0,0.003410,0.002037,0.003371,NaN,0.000473,0.000254,0.000743


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-16,0.001560,0.003629,0.0,0.002005,0.001389,0.001770,0.001662,0.002554,0.000796,0.0,...,0.002575,0.001281,0.001522,0.003443,0.000000,0.001878,-0.000033,0.002676,0.001156,0.002139
2022-02-17,0.001663,0.000000,0.0,0.001421,0.001418,0.001603,0.001648,0.000758,0.001409,0.0,...,0.001949,0.001488,0.001126,0.002233,0.001707,0.001911,-0.000079,0.003084,0.001154,0.001217
2022-02-18,0.001134,0.004881,0.0,0.001558,0.001576,0.001016,0.001649,0.001759,0.001093,0.0,...,0.001930,0.001787,0.001089,0.002373,0.000000,0.001757,0.000000,0.002342,0.000989,0.001508


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                   \
Admin2           Autauga   Baldwin   Barbour     Bibb    Blount   Bullock   
2022-02-16      0.001042  0.001265  0.000000  0.00079  0.001304  0.000882   
2022-02-17      0.001367  0.000897  0.001663  0.00142  0.002604  0.000441   
2022-02-18      0.001430  0.000677  0.000369  0.00063  0.000752  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-02-16      0.000800  0.000640  0.000596  0.001196  ...  0.000903   
2022-02-17      0.000999  0.001119  0.000715  0.001792  ...  0.001353   
2022-02-18      0.001197  0.001661  0.000596  0.001391  ...  0.001201   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-02-16     -0.000534  0.001510 -0.000524   0.001743  0.002183  0.001612   
2022-02-17      0.002672  0.000754  0.000524   0.000549  0.002800  0.000894   
2022-02-18      0.002132  0.001130  0.000524   0.001007  0.002379  0.000179   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-02-16            NaN  0.003495  0.001309  
2022-02-17            NaN  0.000435  0.000654  
2022-02-18            NaN  0.002176  0.000653  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-16,0.005267,0.024237,NaN,0.000294,0.004832,0.003479,0.000773,-0.000194,0.003403,NaN,...,0.000000,0.003511,0.002312,0.000000,0.000000,0.005568,0.013101,0.006099,0.001696,0.0
2022-02-17,0.002141,0.000000,NaN,0.007687,0.004416,0.002663,0.001372,0.000582,0.000377,NaN,...,0.017131,0.003583,0.001845,0.006957,0.009346,0.003859,0.000000,0.006881,0.004003,0.0
2022-02-18,0.005060,0.000000,NaN,0.004197,0.003517,0.003278,0.000514,0.003196,0.001883,NaN,...,0.001760,0.003053,0.003453,0.006908,0.000000,0.003845,0.000000,0.002603,0.001917,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga   Baldwin   Barbour Bibb    Blount Bullock    Butler   
2022-02-16      0.022599  0.001590  0.011111  0.0  0.004695     0.0  0.008929   
2022-02-17      0.000000  0.001587  0.000000  0.0  0.004673     0.0  0.008850   
2022-02-18      0.011050  0.004754  0.010989  0.0  0.000000     0.0  0.000000   

Province_State                                ... Wyoming                  \
Admin2           Calhoun  Chambers  Cherokee  ...    Park Platte Sheridan   
2022-02-16      0.008803  0.000000  0.040541  ...     0.0    0.0      0.0   
2022-02-17      0.001745  0.000000  0.000000  ...     0.0    0.0      0.0   
2022-02-18      0.001742  0.006711  0.000000  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-02-16          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2022-02-17          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2022-02-18          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-16,0.001338,0.001357,0.001642,0.002314,0.000182,0.0,0.004549,0.001852,0.003831,0.003773,...,0.007973,0.001406,4.517509e-21,0.002209,0.012684,0.007912,0.005849,0.001066,0.000841,0.001497
2022-02-17,0.001118,0.001349,0.001534,0.002092,0.000177,0.0,0.002274,0.001874,0.004016,0.005716,...,0.007455,0.001217,2.258755e-21,0.002016,0.013285,0.006249,0.002924,0.001003,0.000935,0.001577
2022-02-18,0.001095,0.001232,0.001367,0.001939,0.000190,0.0,0.001475,0.001811,0.003728,0.006741,...,0.008210,0.001188,1.129377e-21,0.001970,0.014671,0.005467,0.002448,0.001056,0.000934,0.001717


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-16,0.000934,0.001147,0.001522,0.002550,0.000144,NaN,0.003926,0.001597,0.002642,0.013383,...,0.003175,0.001491,0.0,0.001465,0.001951,0.004564,NaN,0.001847,0.000111,0.000479
2022-02-17,0.001199,0.001596,0.001500,0.001275,0.000072,NaN,0.001963,0.001436,0.002779,0.010444,...,0.002907,0.001678,0.0,0.001451,0.002124,0.004068,NaN,0.001160,0.000310,0.000426
2022-02-18,0.000732,0.001089,0.001341,0.000637,0.000036,NaN,0.000981,0.001270,0.002904,0.009877,...,0.002624,0.001459,0.0,0.002431,0.002080,0.003720,NaN,0.000817,0.000282,0.000584


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-16,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-02-17,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-02-18,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-16,0.001462,0.002997,2.980232e-09,0.001488,0.001560,0.001651,0.001740,0.002044,0.000783,1.617950e-212,...,0.002591,0.001448,0.001467,0.002966,0.000632,0.002122,0.000244,0.002525,0.001210,0.001971
2022-02-17,0.001562,0.001498,1.490116e-09,0.001455,0.001489,0.001627,0.001694,0.001401,0.001096,8.089751e-213,...,0.002270,0.001468,0.001297,0.002600,0.001170,0.002017,0.000083,0.002805,0.001182,0.001594
2022-02-18,0.001348,0.003190,7.450581e-10,0.001506,0.001532,0.001321,0.001672,0.001580,0.001094,4.044876e-213,...,0.002100,0.001627,0.001193,0.002486,0.000585,0.001887,0.000041,0.002574,0.001085,0.001551


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-02-16      0.001401  0.001187  0.000248  0.001214  0.001088  0.000684   
2022-02-17      0.001384  0.001042  0.000955  0.001317  0.001846  0.000562   
2022-02-18      0.001407  0.000860  0.000662  0.000974  0.001299  0.000281   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-02-16      0.000875  0.000919  0.000661  0.001361  ...  0.001180   
2022-02-17      0.000937  0.001019  0.000688  0.001577  ...  0.001267   
2022-02-18      0.001067  0.001340  0.000642  0.001484  ...  0.001234   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-02-16      0.000322  0.001462  0.000806   0.002338  0.001958  0.001395   
2022-02-17      0.001497  0.001108  0.000665   0.001444  0.002379  0.001145   
2022-02-18      0.001815  0.001119  0.000595   0.001225  0.002379  0.000662   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-02-16           -1.0  0.002956  0.001366  
2022-02-17           -1.0  0.001696  0.001010  
2022-02-18           -1.0  0.001936  0.000831  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-16,0.004000,0.012194,NaN,0.000440,0.004398,0.002922,0.001202,0.001454,0.002426,NaN,...,0.001667,0.002959,0.003909,0.002319,0.003663,0.005657,0.007397,0.005665,0.002224,0.00332
2022-02-17,0.003071,0.006097,NaN,0.004064,0.004407,0.002792,0.001287,0.001018,0.001402,NaN,...,0.009399,0.003271,0.002877,0.004638,0.006504,0.004758,0.003699,0.006273,0.003114,0.00166
2022-02-18,0.004065,0.003048,NaN,0.004130,0.003962,0.003035,0.000901,0.002107,0.001642,NaN,...,0.005579,0.003162,0.003165,0.005773,0.003252,0.004301,0.001849,0.004438,0.002515,0.00083


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2022-02-16      0.013315  0.002009  0.009267  0.002659  0.002519   
2022-02-17      0.006657  0.001798  0.004634  0.001329  0.003596   
2022-02-18      0.008854  0.003276  0.007811  0.000665  0.001798   

Province_State                                                            ...  \
Admin2               Bullock    Butler   Calhoun      Chambers  Cherokee  ...   
2022-02-16      3.213174e-06  0.004818  0.004527  8.312870e-07  0.021376  ...   
2022-02-17      1.606587e-06  0.006834  0.003136  4.156435e-07  0.010688  ...   
2022-02-18      8.032934e-07  0.003417  0.002439  3.355913e-03  0.005344  ...   

Province_State   Wyoming                                           \
Admin2              Park    Platte  Sheridan  Sublette Sweetwater   
2022-02-16      0.001939  0.000051  0.014797  0.009694   0.006303   
2022-02-17      0.000969  0.000026  0.007398  0.004847   0.003152   
2022-02-18      0.000485  0.000013  0.003699  0.002423   0.001576   

Province_State                                                             
Admin2                 Teton     Uinta Unassigned  Washakie        Weston  
2022-02-16      8.514949e-09  0.000056   0.752941  0.000048  9.536743e-07  
2022-02-17      4.257475e-09  0.000028   0.752941  0.000024  4.768372e-07  
2022-02-18      2.128737e-09  0.000014   0.752941  0.000012  2.384186e-07  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


#### 